# Optimizing Pipelines Project

## Objective
This project is part of the Udacity Azure ML Nanodegree.  
In this project, we build and optimize an Azure ML pipeline using the Python SDK and a provided Scikit-learn model.  
This model is then compared to an Azure AutoML run.  

## 0. Setup

### Initialize workspace and experiment

In [1]:
from azureml.core import Workspace, Experiment

#ws = Workspace.get(name="udacity-project")
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-125472
Azure region: southcentralus
Subscription id: b4a122b5-b4d5-40e7-9878-57b87adf4a8b
Resource group: aml-quickstarts-125472


### Create Cluster

In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

#- Define cluster name
cluster_name = "compute-cluster"

#- Define cluster configuration
cluster_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', 
                                                       max_nodes=4)

#- Create cluster
compute_target = ComputeTarget.create(ws, cluster_name, cluster_config)

#- Set timeout
compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

#- Get cluster status
print(compute_target.get_status().serialize())

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2020-11-09T23:22:28.557000+00:00', 'errors': None, 'creationTime': '2020-11-09T23:22:25.573069+00:00', 'modifiedTime': '2020-11-09T23:22:41.800346+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


## 1. Hyperparameter run

### Create environment

In [3]:
%%writefile conda_dependencies.yml

dependencies:
- python=3.6.2
- scikit-learn
- pip:
  - azureml-defaults

Writing conda_dependencies.yml


In [4]:
from azureml.core import Environment

sklearn_env = Environment.from_conda_specification(name = 'sklearn-env', file_path = './conda_dependencies.yml')

### Define hyperparameter space

In [5]:
#- Import libraries
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

#- These libraries were added
import shutil
from azureml.core import ScriptRunConfig
from azureml.train.hyperdrive import choice

In [6]:
# Specify parameter sampler
#ps = ### YOUR CODE HERE ###
ps = RandomParameterSampling(
    {
        '--C': choice(0.01, 0.1, 1-0, 10.0, 100.0),
        '--max_iter': choice(5000, 10000, 15000)
    }
)

In [7]:
# Specify a Policy
#policy = ### YOUR CODE HERE ###
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

In [8]:
#- Create project folder
if "training" not in os.listdir():
    os.mkdir("./training")

In [9]:
# Create a SKLearn estimator for use with train.py
#est = ### YOUR CODE HERE ###

#- Create project folder variable
project_folder = "./training"

#- Copy train.py file to project folder
shutil.copy('train.py', project_folder)

#- Create estimator
est = ScriptRunConfig(source_directory=project_folder,
                      script='train.py',
                      compute_target=compute_target,
                      environment=sklearn_env)

In [10]:
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
#hyperdrive_config = ### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(run_config=est,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=4)

### Submit and monitor hyperdrive runs

In [11]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
#- Launch hyperparameter tuning job
hyperdrive_run = exp.submit(config=hyperdrive_config)

In [12]:
#- Show details
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [13]:
#- Wait for completion
hyperdrive_run.wait_for_completion(show_output=True)
assert(hyperdrive_run.get_status() == "Completed")

RunId: HD_b3f20a5a-f4c1-4b41-98e7-b2dc346d8720
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_b3f20a5a-f4c1-4b41-98e7-b2dc346d8720?wsid=/subscriptions/b4a122b5-b4d5-40e7-9878-57b87adf4a8b/resourcegroups/aml-quickstarts-125472/workspaces/quick-starts-ws-125472

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-11-09T23:23:21.874371][API][INFO]Experiment created<END>\n""<START>[2020-11-09T23:23:22.576437][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2020-11-09T23:23:22.781189][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2020-11-09T23:23:23.9568034Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_b3f20a5a-f4c1-4b41-98e7-b2dc346d8720
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_b3f20a5a-f4c1-4b41-98e7-b2dc346d8720?wsid=/subscriptions/b4a12

### Images of the hyperparameter runs

![Hyperparameter results 01](images/H_01.png)
![Hyperparameter results 02](images/H_02.png)
![Hyperparameter results 03](images/H_03.png)

### Find and register best model

In [14]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
#- Get best run
best_run_h = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run_h.get_details()['runDefinition']['arguments'])

['--C', '10', '--max_iter', '10000']


In [15]:
#- List model files uploaded
print(best_run_h.get_file_names())

['azureml-logs/55_azureml-execution-tvmps_4ba8ce7d3f3333f203e054e2e93d2612b827cc9e0bff3cb2c0aaafa79b54e7c9_d.txt', 'azureml-logs/65_job_prep-tvmps_4ba8ce7d3f3333f203e054e2e93d2612b827cc9e0bff3cb2c0aaafa79b54e7c9_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_4ba8ce7d3f3333f203e054e2e93d2612b827cc9e0bff3cb2c0aaafa79b54e7c9_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/111_azureml.log', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/engine_spans_l_91f79695-79ba-4952-bedb-0b81410293ea.jsonl', 'logs/azureml/dataprep/python_span_l_91f79695-79ba-4952-bedb-0b81410293ea.jsonl', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/model_h/model_h.joblib']


In [16]:
#- Download saved model
os.makedirs('./model_h', exist_ok=True)

for f in best_run_h.get_file_names():
    if f.startswith('outputs/model_h'):
        output_file_path = os.path.join('./model_h', f.split('/')[-1])
        print('Downloading from {} to {} ...'.format(f, output_file_path))
        best_run_h.download_file(name=f, output_file_path=output_file_path)

## 2. Auto ML run

In [17]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

#- This library was included to load the dataset
from azureml.core import Dataset

#- Define path to data
web_path ='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'

#- Create tabular dataset
ds_ml = Dataset.Tabular.from_delimited_files(path=web_path)

In [18]:
from train import clean_data

# Use the clean_data function to clean your data.
# x, y = clean_data(### YOUR DATA OBJECT HERE ###)
x_ml, y_ml = clean_data(ds_ml)

In [19]:
from sklearn.model_selection import train_test_split

#- Split into 70-30 proportion, since is the general recommended value in the field
#- Set random_state to 0, to ensure that the same random combination is used between runs
x_ml_train, x_ml_val, y_ml_train, y_ml_val = train_test_split(x_ml, y_ml, test_size=0.30, random_state=0)

In [20]:
import pandas as pd

#- Concatenate input dataset and label dataset
training_data = pd.concat ([x_ml_train, y_ml_train], axis=1)
validation_data = pd.concat ([x_ml_val, y_ml_val], axis=1)

In [21]:
#- Create folder to save train data as csv
if not os.path.isdir('data'):
    os.mkdir('data')
    
#- Save the train data to a csv to be uploaded to the datastore
pd.DataFrame(training_data).to_csv("data/train_data.csv", index=False)
pd.DataFrame(validation_data).to_csv("data/val_data.csv", index=False)

ds2 = ws.get_default_datastore()
ds2.upload(src_dir='./data', target_path='bankmarketing', overwrite=True, show_progress=True)


#- Upload the training and validation data as a tabular dataset for access during training on remote compute
training_dataset = Dataset.Tabular.from_delimited_files(path=ds2.path('bankmarketing/train_data.csv'))
validation_dataset = Dataset.Tabular.from_delimited_files(path=ds2.path('bankmarketing/val_data.csv'))

Uploading an estimated of 2 files
Uploading ./data/train_data.csv
Uploaded ./data/train_data.csv, 1 files out of an estimated total of 2
Uploading ./data/val_data.csv
Uploaded ./data/val_data.csv, 2 files out of an estimated total of 2
Uploaded 2 files


In [22]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task= 'classification',
    primary_metric='accuracy',
    training_data=training_dataset,
    label_column_name="y",
    n_cross_validations=2,
    
    debug_log = 'automl_errors.log',
    compute_target=compute_target,
    #validation_data = validation_dataset,
    enable_onnx_compatible_models=True
)

In [23]:
# Submit your automl run

### YOUR CODE HERE ###
remote_run = exp.submit(automl_config, show_output = False)

Running on remote.


In [24]:
remote_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_bd936d41-9163-4487-b720-29647673d06a,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


In [25]:
#- Wait for the remote run to complete
remote_run.wait_for_completion()

{'runId': 'AutoML_bd936d41-9163-4487-b720-29647673d06a',
 'target': 'compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2020-11-09T23:50:47.399534Z',
 'endTimeUtc': '2020-11-10T00:32:20.229686Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'compute-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"b4a122b5-b4d5-40e7-9878-57b87adf4a8b","resource_group":"aml-quickstarts-125472","workspace_name":"quick-starts-ws-125472","region":"southcentralus","compute_target":"compute-cluster","spark_service":null,"azure_service":"remote","_local_managed_run_id":null,"many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":2,"y_m

In [26]:
#- Show results
from azureml.widgets import RunDetails

RunDetails(remote_run).show() 

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

### Images of the AutoML runs

![AutoML results 01](images/A_01.png)
![AutoML results 02](images/A_02.png)

In [27]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
#- Retrieve model explanation
from azureml.core.run import Run

model_explainability_run_id = remote_run.id + "_" + "ModelExplain"
print(model_explainability_run_id)
model_explainability_run = Run(experiment=exp, run_id=model_explainability_run_id)
model_explainability_run.wait_for_completion()

#- Get the best run object
best_run_ml, fitted_model_ml = remote_run.get_output()

#- Print the properties of the model
print (best_run_ml)

AutoML_bd936d41-9163-4487-b720-29647673d06a_ModelExplain
Run(Experiment: udacity-project,
Id: AutoML_bd936d41-9163-4487-b720-29647673d06a_29,
Type: azureml.scriptrun,
Status: Completed)


In [28]:
print (fitted_model_ml)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(allow_chargram=None, enable_dnn=None,
                                 enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedso...
                                                                                                  min_samples_split=0.01,
                                                                                                  min_weight_fraction_leaf=0.0,
                                                                                                  n_estimato

In [29]:
#- Retrieve best ONNX model
best_run_onnx, onnx_mdl = remote_run.get_output(return_onnx_model=True)

In [31]:
from azureml.automl.runtime.onnx_convert import OnnxConverter

#- Save best ONNX model
onnx_fl_path = "./best_model.onnx"
OnnxConverter.save_onnx_model(onnx_mdl, onnx_fl_path)

## 3. Cluster clean up

In [32]:
#- Delete compute target
compute_target.delete()